# Production Machine Learning Systems

This notebook covers the complete lifecycle of deploying machine learning models to production, including monitoring, scaling, and maintenance.

## Model Deployment Strategies

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import json
from datetime import datetime

# Create a sample model for deployment
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, 'production_model.pkl')
print("Model saved successfully!")

In [ ]:
# Create model metadata
model_metadata = {
    "model_name": "production_classifier",
    "version": "1.0.0",
    "created_date": datetime.now().isoformat(),
    "features": [f"feature_{i}" for i in range(10)],
    "target": "binary_classification",
    "performance": {
        "accuracy": model.score(X_test, y_test),
        "n_features": 10,
        "n_classes": 2
    },
    "requirements": [
        "scikit-learn>=1.0.0",
        "numpy>=1.20.0",
        "pandas>=1.3.0"
    ]
}

with open('model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2)

print("Model metadata saved!")
print(json.dumps(model_metadata, indent=2))

## Model Monitoring and Drift Detection

In [ ]:
class ModelMonitor:
    def __init__(self, model_path, metadata_path):
        self.model = joblib.load(model_path)
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
        self.predictions_log = []
        self.performance_log = []
    
    def predict(self, X):
        prediction = self.model.predict(X)
        probability = self.model.predict_proba(X)
        
        # Log prediction
        self.predictions_log.append({
            'timestamp': datetime.now().isoformat(),
            'prediction': int(prediction[0]),
            'confidence': float(np.max(probability)),
            'features': X.tolist()
        })
        
        return prediction, probability
    
    def check_drift(self, new_data, threshold=0.1):
        """Simple drift detection using statistical comparison"""
        if len(self.predictions_log) < 100:
            return False, "Insufficient data for drift detection"
        
        # Compare feature distributions
        recent_features = [pred['features'] for pred in self.predictions_log[-100:]]
        recent_mean = np.mean(recent_features, axis=0)
        new_mean = np.mean(new_data, axis=0)
        
        drift_score = np.mean(np.abs(recent_mean - new_mean))
        
        if drift_score > threshold:
            return True, f"Drift detected with score: {drift_score:.3f}"
        
        return False, f"No drift detected. Score: {drift_score:.3f}"
    
    def get_performance_report(self):
        if not self.predictions_log:
            return "No predictions logged yet"
        
        confidences = [pred['confidence'] for pred in self.predictions_log]
        predictions = [pred['prediction'] for pred in self.predictions_log]
        
        return {
            'total_predictions': len(predictions),
            'avg_confidence': np.mean(confidences),
            'min_confidence': np.min(confidences),
            'prediction_distribution': {
                'class_0': predictions.count(0),
                'class_1': predictions.count(1)
            }
        }

# Initialize monitor
monitor = ModelMonitor('production_model.pkl', 'model_metadata.json')
print("Model monitor initialized!")

In [ ]:
# Simulate production predictions
print("Simulating production predictions...")

for i in range(50):
    # Simulate new data
    new_sample = np.random.randn(1, 10)
    pred, prob = monitor.predict(new_sample)
    
    if i % 10 == 0:
        print(f"Prediction {i+1}: {pred[0]} (confidence: {np.max(prob):.3f})")

# Get performance report
report = monitor.get_performance_report()
print("\nPerformance Report:")
print(json.dumps(report, indent=2))

## A/B Testing for Models

In [ ]:
class ABTestManager:
    def __init__(self):
        self.models = {}
        self.traffic_split = {}
        self.results = {}
    
    def add_model(self, name, model_path, traffic_percentage):
        self.models[name] = joblib.load(model_path)
        self.traffic_split[name] = traffic_percentage / 100.0
        self.results[name] = []
    
    def get_model_for_request(self):
        rand = np.random.random()
        cumulative = 0
        
        for model_name, percentage in self.traffic_split.items():
            cumulative += percentage
            if rand <= cumulative:
                return model_name
        
        return list(self.models.keys())[-1]  # fallback
    
    def predict(self, X):
        model_name = self.get_model_for_request()
        model = self.models[model_name]
        prediction = model.predict(X)[0]
        
        # Log result
        self.results[model_name].append({
            'timestamp': datetime.now().isoformat(),
            'prediction': int(prediction),
            'features': X.tolist()
        })
        
        return prediction, model_name
    
    def get_results(self):
        results_summary = {}
        for model_name, results in self.results.items():
            predictions = [r['prediction'] for r in results]
            results_summary[model_name] = {
                'total_requests': len(predictions),
                'class_distribution': {
                    '0': predictions.count(0),
                    '1': predictions.count(1)
                },
                'traffic_percentage': self.traffic_split[model_name] * 100
            }
        
        return results_summary

# Create a second model for comparison
model2 = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=123)
model2.fit(X_train, y_train)
joblib.dump(model2, 'model_v2.pkl')

# Set up A/B test
ab_test = ABTestManager()
ab_test.add_model('model_v1', 'production_model.pkl', 70)  # 70% traffic
ab_test.add_model('model_v2', 'model_v2.pkl', 30)  # 30% traffic

print("A/B test setup complete!")

In [ ]:
# Run A/B test simulation
print("Running A/B test simulation...")

for i in range(100):
    sample = np.random.randn(1, 10)
    prediction, model_used = ab_test.predict(sample)
    
    if i % 20 == 0:
        print(f"Request {i+1}: {prediction} (model: {model_used})")

# Get A/B test results
ab_results = ab_test.get_results()
print("\nA/B Test Results:")
print(json.dumps(ab_results, indent=2))

## Key Production Considerations

### 1. **Model Versioning**
- Track model versions with semantic versioning
- Store model artifacts and metadata
- Maintain backward compatibility

### 2. **Monitoring**
- Track prediction confidence scores
- Monitor data drift
- Log performance metrics
- Set up alerting for anomalies

### 3. **A/B Testing**
- Gradual rollout of new models
- Statistical significance testing
- Traffic splitting strategies
- Rollback capabilities

### 4. **Scalability**
- Horizontal scaling with load balancers
- Container orchestration (Kubernetes)
- Serverless deployment options
- Caching strategies

### 5. **Security**
- API authentication and authorization
- Data encryption
- Input validation
- Rate limiting

## Production Deployment Checklist

### Pre-Deployment:
- [ ] Model performance meets requirements
- [ ] Comprehensive testing completed
- [ ] Documentation is complete
- [ ] Monitoring is configured
- [ ] Rollback plan is ready

### Deployment:
- [ ] Model version is tagged
- [ ] Configuration is validated
- [ ] Health checks are passing
- [ ] Traffic is routed correctly
- [ ] Monitoring is active

### Post-Deployment:
- [ ] Performance is monitored
- [ ] Logs are collected
- [ ] Alerts are configured
- [ ] User feedback is gathered
- [ ] Model is updated based on feedback